<a href="https://colab.research.google.com/github/asraj/END/blob/main/END_session9_model1_Customer_Support_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Customer Support on Twitter: This Kaggle dataset includes more than 3 million tweets and responses from leading brands on Twitter.

## **Data Preparation**
The data is available as csv file in Kaggle. The following code

1) downloads the zip file from kaggle

2) extract the contents

3) loads csv file as pandas DataFrame 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls /content/gdrive/MyDrive/kaggle

kaggle.json  sample.csv  twcs  tweets.tsv


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [4]:
!pwd

/content


In [5]:
%cd /content/gdrive/MyDrive/kaggle

/content/gdrive/MyDrive/kaggle


In [38]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 93% 157M/169M [00:01<00:00, 107MB/s] 
100% 169M/169M [00:01<00:00, 105MB/s]


In [39]:
!unzip \*.zip  && rm *.zip

Archive:  customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [42]:
!ls twcs

twcs.csv


In [6]:
import pandas as pd

In [7]:
# read data
df = pd.read_csv('/content/gdrive/MyDrive/kaggle/twcs/twcs.csv')
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


Filter relevant tweets and combine tweets into Q&A

In [8]:
# Get tweets that are first inbound tweets to companies
first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]

# Get replies for first inbound tweets
inbound_and_replies = pd.merge(first_inbound, df, left_on='tweet_id', right_on='in_response_to_tweet_id')

# Remove replies that aren't from companies
inbound_and_replies = inbound_and_replies[inbound_and_replies.inbound_y ^ True]

# Replace carriage returns
inbound_and_replies = inbound_and_replies.replace('\r', '', regex=True)

In [9]:
# Extract inbound tweets and responses
tweets = inbound_and_replies[['text_x', 'text_y']]
tweets.columns = ['tweet', 'response']
tweets.head()

,tweet,response
0,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
1,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...
2,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...
3,@115714 y’all lie about your “great” connectio...,@115713 H there! We'd definitely like to work ...
4,"@115714 whenever I contact customer support, t...",@115715 Please send me a private message so th...


In [10]:
tweets.shape

(794299, 2)

In [33]:
#Use a smaller subset that is managable
tweets.sample(n=20_000).to_csv("tweets.tsv", index=False, sep='\t')

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
import numpy as np

import random
import math
import time

In [35]:
# Use same field for both columns since they have a shared vocabulary
TWEET = Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('src', TWEET), ('trg', TWEET)]

In [36]:
# Create dataset
dataset = TabularDataset(
    path='tweets.tsv',
    format='TSV',
    fields=fields,
    skip_header=True
)

In [37]:
# Train/val split
train_data, valid_data, test_data = dataset.split(split_ratio=[0.75, 0.15, 0.1])
print(len(train_data), len(valid_data), len(test_data))

15000 2000 3000


In [38]:
print(vars(train_data.examples[0]))

{'src': ['@safaricom_care', 'just', 'received', 'a', 'fraudulent', 'call', 'from', 'this', 'number', '0727', '964885', 'block', 'asap'], 'trg': ['@255763', ' ', 'we', "'ve", 'shared', 'the', 'number', 'for', 'https://t.co/9tum7nuotx', 'can', 'always', 'share', 'such', 'with', 'our', 'fraud', 'team', 'on', '333', '(', 'free).^nr']}


In [39]:
#Build the vocabulary.
TWEET.build_vocab(train_data, min_freq = 2)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
device

device(type='cuda')

In [42]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort_key = lambda x : len(x.src), 
    batch_size = BATCH_SIZE, 
    device = device)

In [43]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [44]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [45]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [46]:
INPUT_DIM = len(TWEET.vocab)
OUTPUT_DIM = len(TWEET.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [47]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(13260, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(13260, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=13260, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,927,052 trainable parameters


In [49]:
optimizer = optim.Adam(model.parameters())

In [50]:
PAD_IDX = TWEET.vocab.stoi[TWEET.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [51]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [52]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [53]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [54]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 29s
	Train Loss: 5.849 | Train PPL: 346.931
	 Val. Loss: 5.510 |  Val. PPL: 247.265
Epoch: 02 | Time: 1m 30s
	Train Loss: 5.063 | Train PPL: 158.113
	 Val. Loss: 5.727 |  Val. PPL: 307.125
Epoch: 03 | Time: 1m 30s
	Train Loss: 4.821 | Train PPL: 124.031
	 Val. Loss: 5.685 |  Val. PPL: 294.555
Epoch: 04 | Time: 1m 30s
	Train Loss: 4.630 | Train PPL: 102.519
	 Val. Loss: 5.693 |  Val. PPL: 296.789
Epoch: 05 | Time: 1m 30s
	Train Loss: 4.474 | Train PPL:  87.709
	 Val. Loss: 5.739 |  Val. PPL: 310.618
Epoch: 06 | Time: 1m 30s
	Train Loss: 4.329 | Train PPL:  75.842
	 Val. Loss: 5.697 |  Val. PPL: 298.054
Epoch: 07 | Time: 1m 30s
	Train Loss: 4.123 | Train PPL:  61.748
	 Val. Loss: 5.853 |  Val. PPL: 348.379
Epoch: 08 | Time: 1m 30s
	Train Loss: 4.008 | Train PPL:  55.033
	 Val. Loss: 5.859 |  Val. PPL: 350.414
Epoch: 09 | Time: 1m 30s
	Train Loss: 3.957 | Train PPL:  52.312
	 Val. Loss: 5.774 |  Val. PPL: 321.950
Epoch: 10 | Time: 1m 30s
	Train Loss: 3.799 | Train PPL